In [52]:
# Importando los módulos que necesitaremos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from matplotlib.ticker import PercentFormatter
from datetime import datetime
import os

# Configurando los estílos de los gráficos
plt.ioff()
sns.set_context('talk')
sns.set_style("whitegrid")

# Definiendo Constantes
BINS = [0, 10, 20, 30, 40, 50, 60, 70, 80, 300]

## Limpieza de Datos

In [53]:
# Leo la fuente de datos local en CSV 
df = pd.read_csv('data/vacunas_covid.csv')
df.head()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20210227,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
1,20210227,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210219,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
2,20210227,54048fe3b06c35f14350d7e81340bdbd,PERSONAL DE SALUD,44.0,MASCULINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
3,20210227,746ad64c513b5d53b987ed48a6b45f77,PERSONAL DE SALUD,54.0,FEMENINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
4,20210227,80f0a41dd63af3b96d4a80bfab2eaa61,PERSONAL DE SALUD,35.0,FEMENINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO


In [54]:
# Obtengo el conteo del número de valores en cada columnas
df.count()

FECHA_CORTE         265608
UUID                265608
GRUPO_RIESGO        265608
EDAD                265502
SEXO                264719
FECHA_VACUNACION    265608
DOSIS               265608
FABRICANTE          265608
DIRESA              265608
DEPARTAMENTO        265608
PROVINCIA           265608
DISTRITO            265608
dtype: int64

In [55]:
# Verificamos que las columnas DEPARTAMENTO y SEXO tengan los valores indicados
print(df['DEPARTAMENTO'].unique())
print(df['SEXO'].unique())

['LAMBAYEQUE' 'LIMA' 'AMAZONAS' 'LA LIBERTAD' 'CAJAMARCA' 'AYACUCHO'
 'APURIMAC' 'SAN MARTIN' 'PIURA' 'HUANUCO' 'LORETO' 'CALLAO' 'CUSCO' 'ICA'
 'UCAYALI' 'HUANCAVELICA' 'AREQUIPA' 'JUNIN' 'MADRE DE DIOS' 'PASCO'
 'PUNO' 'ANCASH' 'MOQUEGUA' 'TACNA' 'TUMBES']
['MASCULINO' 'FEMENINO' nan]


In [56]:
# Filtramos las filas que no tienen valor en las columnas EDAD o SEXO
df = df[df['EDAD'].notnull() & df['SEXO'].notnull()]

## Transformación de Datos

In [57]:
# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y EDAD para el DataFrame
gb = df[["DEPARTAMENTO","SEXO",'EDAD']].groupby(['DEPARTAMENTO', 'SEXO', pd.cut(df['EDAD'], BINS, False)])

In [58]:
def get_data_by_department(department):
    return df[df['DEPARTAMENTO'] == department]

## Visualización de Datos

In [59]:
# Visualizo los datos agrupados via DataFrame
gb.size().unstack()

EDAD                     [0, 10)  [10, 20)  [20, 30)  [30, 40)  [40, 50)  \
DEPARTAMENTO  SEXO                                                         
AMAZONAS      FEMENINO         0         7       612      1060       619   
              MASCULINO        0         4       303       590       419   
ANCASH        FEMENINO         0         1       986      1772      1328   
              MASCULINO        0         6       393       860       683   
APURIMAC      FEMENINO         0         3       741      1380      1051   
              MASCULINO        0         3       275       670       662   
AREQUIPA      FEMENINO         0         4      1571      2935      2258   
              MASCULINO        0         4       532      1156       977   
AYACUCHO      FEMENINO         0         4       871      1567      1171   
              MASCULINO        0         4       308       750       750   
CAJAMARCA     FEMENINO         0         2      1243      2052      1320   
              MASCULINO        0         8       522      1075       741   
CALLAO        FEMENINO         0         5       941      2298      1930   
              MASCULINO        0         8       536      1075       961   
CUSCO         FEMENINO         0         9      1005      2508      1665   
              MASCULINO        0         9       414      1039       922   
HUANCAVELICA  FEMENINO         0         5       820      1176       685   
              MASCULINO        0         4       307       546       431   
HUANUCO       FEMENINO         0         4       924      1587      1044   
              MASCULINO        0        11       349       830       658   
ICA           FEMENINO         0         3      1166      1619      1152   
              MASCULINO        0         5       394       719       578   
JUNIN         FEMENINO         0         3      1807      2561      1544   
              MASCULINO        0         7       518      1056       727   
LA LIBERTAD   FEMENINO         0         4      1732      3184      2127   
              MASCULINO        0        10       800      1457      1198   
LAMBAYEQUE    FEMENINO         0         1      1143      2016      1522   
              MASCULINO        0         1       441       896       792   
LIMA          FEMENINO         0        39      7845     19744     15353   
              MASCULINO        0        61      3855      9214      7628   
LORETO        FEMENINO         0         2       516      1204       829   
              MASCULINO        0         6       376       768       520   
MADRE DE DIOS FEMENINO         0         4       174       355       259   
              MASCULINO        0         1        75       174       129   
MOQUEGUA      FEMENINO         0         3       418       735       627   
              MASCULINO        0         4       133       336       274   
PASCO         FEMENINO         0         1       428       612       449   
              MASCULINO        0         3       159       302       291   
PIURA         FEMENINO         0         7      1338      2464      2164   
              MASCULINO        0        11       607      1177      1079   
PUNO          FEMENINO         0         3       827      1611      1035   
              MASCULINO        0         0       273       695       599   
SAN MARTIN    FEMENINO         0         9       936      1669       836   
              MASCULINO        0        11       529      1019       581   
TACNA         FEMENINO         0         4       415       741       704   
              MASCULINO        0         6       222       331       322   
TUMBES        FEMENINO         0         0       314       512       428   
              MASCULINO        0         0       171       277       248   
UCAYALI       FEMENINO         0         3       562       893       649   
              MASCULINO        0         6       333       511       407   

EDAD                     [50, 60)  [60, 70)  [7

In [60]:
yticks = [5,15,25,35,45,55,65,75,85]

def generate_plot(department, data, date_range):

    # Creamos la figura, los ejes y agregamos la atribución
    fig, axs = plt.subplots(1,2, figsize=(20,10))
    plt.figtext(0.1,0.95, department + ': DISTRIBUCIÓN DE VACUNADOS CONTRA COVID-19 (N=' + str(len(data)) + ')', ha='left', fontsize=24)
    plt.figtext(0.1,0.91, 'Rango de fechas de vacunación: del {} al {}'.format(f'{date_range[0]:%Y-%m-%d}',f'{date_range[1]:%Y-%m-%d}'), ha='left', fontsize=20, color='#999')
    plt.figtext(0.9,0.02, """Fuente: https://www.datosabiertos.gob.pe/dataset/vacunaci%C3%B3n-contra-covid-19-ministerio-de-salud-minsa
    https://malexandersalazar.github.io/, Moisés Alexander Salazar Vila, """ + f'{datetime.now():%Y-%m-%d}', ha='right')

    ## 1. Trazamos el primer histograma
    N, bins, patches = axs[0].hist(data["EDAD"], BINS, orientation = 'horizontal')

    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    fracs = N / N.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[0].set_yticklabels([])
    axs[0].yaxis.set_tick_params(width=0)
    axs[0].set_yticks(yticks, minor=True)
    axs[0].set_yticklabels(['0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+'], minor=True)
    axs[0].set_ylim(0,90)
    axs[0].set_ylabel('Grupo etáreo (años)')

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0].text(N[i]+15, yticks[i]-1, str(round(N[i]*100/len(data),2)) + '%')

    ## 2. Trazamos el segundo histograma
    males = data[data['SEXO'] == 'MASCULINO']["EDAD"]
    females = data[data['SEXO'] == 'FEMENINO']["EDAD"]

    N, bins, patches = axs[1].hist([males, females], BINS, orientation = 'horizontal')

    axs[1].set_yticklabels([])
    axs[1].yaxis.set_tick_params(width=0)
    axs[1].set_yticks(yticks, minor=True)
    axs[1].set_ylim(0,90)

    for i in range(len(BINS)-1):
        axs[1].text(N[0][i]+15, yticks[i]-3, str(round(N[0][i]*100/len(data),2)) + '%')
        axs[1].text(N[1][i]+15, yticks[i]+1, str(round(N[1][i]*100/len(data),2)) + '%')

    axs[1].legend(['Hombres (n=' + str(len(males)) + ')', 'Mujeres (n=' + str(len(females)) + ')'])

    ## 3. Mostramos, guardamos y generamos el Markdown para las imágenes

    sns.despine(left=False, bottom=False)
    filename = 'dist/{}_{}.png'.format(f'{datetime.now():%Y%m%d}', department.replace(' ', '_'))
    plt.savefig("../" + filename)
    with open("../dist/images.txt", "a") as f:
        f.write('### {}\n'.format(department))
        f.write('![alt text]({} "{}")\n'.format(filename, department.replace(' ', '_')))

    # plt.show()

In [61]:
if os.path.exists("../dist/images.txt"):
  os.remove("../dist/images.txt")

departments_ordered = np.sort(df['DEPARTAMENTO'].unique())
for d in departments_ordered:
    data = get_data_by_department(d)
    dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
    generate_plot(d, data, (dates.min(), dates.max()))